In [15]:
from pprint import pprint
from llama_index.core.agent import FunctionCallingAgent
from load_from_chroma import get_index_from_chroma
import logging
# from llama_index.llms.deepseekai import DeepSeek
from llama_index.llms.glm import GLM

from llama_index.core import Settings
from Embedding import BGEEbedding

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
Settings.embed_model = BGEEbedding()

In [3]:
# BGEEbedding().get_text_embedding("""你好？""")

[-0.03454579785466194,
 0.018909309059381485,
 -0.05603305250406265,
 -0.02396826446056366,
 -0.006952853873372078,
 -0.05987708643078804,
 0.0003675141779240221,
 -0.03530440106987953,
 -0.013850042596459389,
 0.01828792132437229,
 0.0038161540869623423,
 0.025425385683774948,
 -0.0018067427445203066,
 -0.005545476917177439,
 0.00994128081947565,
 -0.021761341020464897,
 -0.005273332819342613,
 -0.03914815932512283,
 -0.0012348685413599014,
 -0.04105328768491745,
 -0.030950045213103294,
 -0.016969867050647736,
 0.0071548898704349995,
 0.006588758435100317,
 0.029201572760939598,
 0.031024504452943802,
 -0.0550423264503479,
 0.0026396699249744415,
 0.029942600056529045,
 -0.001450795796699822,
 0.039821166545152664,
 0.004922904074192047,
 0.003823748789727688,
 -0.04000212997198105,
 -0.006599328014999628,
 -0.039164166897535324,
 0.01058820728212595,
 -0.025377998128533363,
 -0.038142699748277664,
 0.028079526498913765,
 0.029561355710029602,
 0.024563610553741455,
 0.030786393210291

In [16]:
model = GLM()

In [17]:
model.complete("""你好""")

INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"


CompletionResponse(text='你好👋！我是人工智能助手智谱清言（ChatGLM），很高兴见到你，欢迎问我任何问题。', additional_kwargs={}, raw=ChatCompletion(id='20240924110407d644aa5153534da3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='你好👋！我是人工智能助手智谱清言（ChatGLM），很高兴见到你，欢迎问我任何问题。', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1727147050, model='glm-4-0520', object=None, service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=28, prompt_tokens=6, total_tokens=34, completion_tokens_details=None), request_id='20240924110407d644aa5153534da3'), logprobs=None, delta=None)

In [9]:
print(resp)

你好！很高兴见到你。有什么我可以帮忙的吗？


In [14]:
pprint(resp.raw.choices[0].message)

ChatCompletionMessage(content='你好！很高兴见到你。有什么我可以帮忙的吗？', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [19]:
law_index = get_index_from_chroma(path=""".\\LawDb""",collection="""laws""")

In [20]:

embedding_model =  BGEEbedding()

Settings.embed_model=embedding_model

In [21]:
l = law_index.as_query_engine(llm=model)

In [22]:
l.query("""故意杀人判几年""")

INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"


Response(response='故意杀人的，处死刑、无期徒刑或者十年以上有期徒刑；情节较轻的，处三年以上十年以下有期徒刑。', source_nodes=[NodeWithScore(node=TextNode(id_='75fe2b61-2b14-4a92-9531-843a418fbbc7', embedding=None, metadata={'file_path': 'd:\\code\\law_llama_system\\Law-Book\\7-刑法\\刑法.md', 'file_name': '刑法.md', 'file_size': 217091, 'creation_date': '2024-09-13', 'last_modified_date': '2024-01-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b7538175-d66f-43f8-b33e-d6e13b319d8f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'd:\\code\\law_llama_system\\Law-Book\\7-刑法\\刑法.md', 'file_name': '刑法.md', 'file_size': 217091, 'creation_date': '2024-09-13', 'last_modified_date': '2024-01-22'}, hash='90179f79a2eb885a1f8dd547ea4748e010021257f6

In [24]:
from llama_index.core.tools import QueryEngineTool,ToolMetadata


tool = QueryEngineTool(
    query_engine=law_index.as_query_engine(llm=model),
    metadata=ToolMetadata(
        name="""law_retriever""",
        description="""
        当用户询问的是法律相关的问题时被调用"""
    )
)



In [33]:
agent = FunctionCallingAgent.from_tools(
    [tool],
    llm=model,
    verbose=True,
)

In [39]:
import json
l = json.loads("""{"input": "\u6740\u4eba\u72af\u4ec0\u4e48\u6cd5\uff1f"}""")

In [40]:
l

{'input': '杀人犯什么法？'}

In [23]:
res = agent.chat("""杀人犯什么法？""")

NameError: name 'agent' is not defined

In [37]:
pprint(res)

AgentChatResponse(response='故意杀人罪是刑法中的一个重要罪名，根据刑法规定，故意杀人的，处死刑、无期徒刑或者十年以上有期徒刑；情节较轻的，处三年以上十年以下有期徒刑。这一规定体现了我国法律对生命权的尊重和保护，同时也体现了对故意杀人行为的严厉惩处。', sources=[ToolOutput(content='杀人犯刑法中的故意杀人罪。根据刑法规定，故意杀人的，处死刑、无期徒刑或者十年以上有期徒刑；情节较轻的，处三年以上十年以下有期徒刑。', tool_name='law_retriever', raw_input={'input': '杀人犯什么法？'}, raw_output=Response(response='杀人犯刑法中的故意杀人罪。根据刑法规定，故意杀人的，处死刑、无期徒刑或者十年以上有期徒刑；情节较轻的，处三年以上十年以下有期徒刑。', source_nodes=[NodeWithScore(node=TextNode(id_='9048b1cc-2fcb-4c58-8ff0-0bea98b6fe50', embedding=None, metadata={'file_path': 'D:\\code\\law_llama_system\\Law-Book\\7-刑法\\刑法.md', 'file_name': '刑法.md', 'file_size': 217091, 'creation_date': '2024-09-13', 'last_modified_date': '2024-01-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNo

In [ ]:
agent = ReActAgent.from_tools(
    tools=[tool],
    llm=llm,
    verbose=True,
)

In [ ]:
agent.chat("""今天天气怎么样？""")

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker,StructuredPlannerAgent

worker_agent = FunctionCallingAgentWorker.from_tools(
    tools=individual_query_engine_tools,
    llm_model=llm_model,
)

agent = StructuredPlannerAgent(worker_agent)